Hill Cipher
======

1. [Introduction](#introduction)
2. [Modular arithmetic](#modular_arithmetic)
   - [Multiplication and division](#multiplication_and_division)
   - [Matrices](#matrices)
3. [Hill Cipher method](#hill_cipher)
  - [Alphabet](#alphabet)
  - [Strings <-> numerical vectors](#strings_vectors)
  - [Encoding](#encoding)
  - [Inverting the key matrix and decoding](#key_matrix)
  - [Extras](#extras)
4. [Breaking the cipher](#hacking)
5. [All the code](#allcode)

<a id="introduction"></a>
Introduction
-------------

The Hill Cipher is a cipher that uses linear algebra and modular arithmetic to encrypt messages. The message you want to encrypt is split into chunks of size $n$ and those chunks are turned into vectors of integers. Then you multiply every vector by your secret $n \times n$ matrix and you put everything together, to make the encrypted message.

For this we will fix the alphabet as _" abcdefghijklmnopqrstuvwxyz,."_ (notice the blank space in the beginning) which allows us to get a correspondence between the characters in the alphabet and the numbers $0$ through $28$.

If you want to encrypt the message "cake", assuming we have the key matrix M

$$M = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}$$

all we have to do is turn "cake" into $(3,1,11,5)$ and then multiply the key matrix $M$ with the column vectors $[3, 1]^T$ and $[11, 5]^T$, which gives

$$M[3, 1]^T = [5, 13]^T,\ M[11, 5]^T = [21, 53]^T$$

then we take everything modulo $29$, which gives $[5, 13] \equiv [5, 13]$ and $[21, 53] \equiv [21, 24] \mod 29$ and to finish it up we convert the numbers back into letters, giving $(5,13,21,24) \mapsto$ "emux"

If we wanted to decrypt the message, we just needed to find the inverse of $M$ modulo $29$, which happens to be

$$M^{-1} = \begin{bmatrix} 27 & 1 \\ 16 & 14 \end{bmatrix}$$

and we would repeat the whole process but with $M^{-1}$ instead of $M$.

<a id="modular_arithmetic"></a>
Modular Arithmetic
--------------------

Modular arithmetic is pretty similar to regular arithmetic, but we do all the calculations modulo $m$. In a simplified manner, we want all numbers to be in the set $\{0, 1, 2, \cdots, m-1\}$ which we will call $\mathbb{Z}_m$. To keep all the numbers inside $\mathbb{Z}_m$, we can add/subtract multiples of $m$ whenever needed.

Modular arithmetic is around us in our daily lives, even though people don't notice! The way we deal with the hours of the day is by making calculations in $\mathbb{Z}_{24}$: suppose now it is 13h. What will be the time from 14 hours from now? $13 + 14 = 27$ but of course it won't be 27h because a day only has 24h. But we know that $27 \equiv 3 \mod 24$ hence it will be 3 in the morning.

Some other examples include
$$\begin{align}
3 \equiv 1 &\mod{2} \\
15 \equiv 3 &\mod{12} \\
-1 \equiv 4 &\mod{5} \\
102 \equiv 0 &\mod{3}
\end{align}$$

In Python, we can find the correspondent of a number $n$ in $\mathbb{Z}_m$ by typing `n % m` which is also the remainder of the division of $n$ by $m$.

In [129]:
print(3 % 2)
print(15 % 12)
print(-1 % 5)
print(102 % 3)

1
3
4
0


<a id="multiplication_and_division"></a>
### Multiplication and division

When we perform multiplications, we can deal with each of the factors separately. In fact,
$$5 \times 13 \equiv 2 \times 13 \equiv 2 \times 1 \mod{3}$$
because we have that $5 \equiv 2 \mod{3}$ and $13 \equiv 1 \mod{3}$.

The only operation that requires a bit more caution is division. Recall that dividing by $d$ is the same as multiplying by $\frac1d = d^{-1}$, the inverse of $d$. If we manage to characterize the "inverse of $d$" we may be able to generalize it to modular arithmetic. The inverse of $d$ is a special number $q$ (which we denote by $d^{-1}$) with the amazing property that $qd = dq = 1$. In modular arithmetic, we will define the inverse of a number $n$ as the number $n^{-1}$ such that $n^{-1}n = nn^{-1} \equiv 1 \mod{m}$

For example, the inverse of $2$ modulo $3$ is $2$, because $2\times2 = 4 \equiv 1 \mod{3}$. Some other examples include $10^{-1} = 4 \mod{13}$, $5^{-1} = 5 \mod{6}$ and $7^{-1} = 10 \mod{23}$.

When we are dealing with regular division in $\mathbb{R}$, the only number that does not have an inverse is $0$. When we are dealing with modular arithmetic in $\mathbb{Z}_m$, $0$ doesn't have an inverse. But it may happen that, in $\mathbb{Z}_m$, other numbers cannot have an inverse. Take $\mathbb{Z}_4$ as an example; we will see now that $2$ cannot have an inverse in $\mathbb{Z}_4$: let us call $n$ to the potential inverse of $2$, then $2n \equiv 1 \mod{4}$. But regardless of the $n$ we choose, $2n$ will always be an even number, while $x \equiv 1 \mod{4}$ has to be odd. (In fact if a number $x$ is equivalent to $1$ modulo $m$, then $x$ has to be of the form $x = km + 1$ for some integer $k$.)

For people that are not familiar with modular arithmetic this might be a lot to take in. Just take into account that inverting numbers in $\mathbb{Z}_m$ may be really annoying. The following two rules apply:
 - If the $m$ in $\mathbb{Z}_m$ is prime, we usually write $\mathbb{Z}_p$ and **all** numbers except $0$ have an inverse;
 - If $m$ is not prime, the numbers in $\mathbb{Z}_m$ that you **can** invert are the ones who do **not** share common divisors with $m$.

For this notebook we will write our own function that finds inverses in $\mathbb{Z}_m$. We will decide that if a number $n$ does not have an inverse in $\mathbb{Z}_m$ then the function will return $0$; otherwise it will return its inverse.

In [130]:
def inversemod(n: int, mod: int) -> int:
    for i in range(1, mod+1):
        if (n*i % mod) == 1:
            return i
    else:
        return 0

We will use this function to verify that the inverses I stated above are, in fact, inverses. I will also show that in $\mathbb{Z}_{64}$ the invertible numbers are the odd ones (according to the rule, the numbers that can be inverted in $\mathbb{Z}_{64}$ are the ones who don't share divisors with $64 = 2^6$, i.e. the odd numbers.

In [131]:
print(inversemod(2, 3) == 2)
print(inversemod(10, 13) == 4)
print(inversemod(5, 6) == 5)
print(inversemod(7, 23) == 10)
print(list(map(lambda n: inversemod(n, 64), [i for i in range(64)])))

True
True
True
True
[0, 1, 0, 43, 0, 13, 0, 55, 0, 57, 0, 35, 0, 5, 0, 47, 0, 49, 0, 27, 0, 61, 0, 39, 0, 41, 0, 19, 0, 53, 0, 31, 0, 33, 0, 11, 0, 45, 0, 23, 0, 25, 0, 3, 0, 37, 0, 15, 0, 17, 0, 59, 0, 29, 0, 7, 0, 9, 0, 51, 0, 21, 0, 63]


In [132]:
import numpy as np

### Matrices

Working with matrices can be done in the usual way, but in the end of additions/subtractions/multiplications we just take everything modulo $m$.

We will need to invert matrices in our method. Usually, the matrices need to have non-zero determinant in order for them to be invertible. In $\mathbb{Z}_m$, there is a similar criterion: a matrix $M$ is invertible in $\mathbb{Z}_m$ if $\det{M} \not\equiv 0 \mod{m}$ and the Gauss-Jordan method still works in $\mathbb{Z}_m$.

Below I show a couple of matrix operations modulo $29$.

In [133]:
M = np.array([[1,2],[3,4]])
Minv = np.array([[27,1],[16,14]])

print(np.dot(M, Minv))
print(np.dot(M, Minv) % 29)

identity = np.eye(4)
A = np.zeros([4,4])
A[0,0] = 22
A[1,1] = 1
A[2,2] = 3
A[3,3] = 6
print(A)
print(A + A + A)
print((A+A+A) % 29)

[[ 59  29]
 [145  59]]
[[1 0]
 [0 1]]
[[ 22.   0.   0.   0.]
 [  0.   1.   0.   0.]
 [  0.   0.   3.   0.]
 [  0.   0.   0.   6.]]
[[ 66.   0.   0.   0.]
 [  0.   3.   0.   0.]
 [  0.   0.   9.   0.]
 [  0.   0.   0.  18.]]
[[  8.   0.   0.   0.]
 [  0.   3.   0.   0.]
 [  0.   0.   9.   0.]
 [  0.   0.   0.  18.]]


<a id="hill_cipher"></a>
Hill Cipher method
------------------

To refresh your memory, the method of the Hill Cipher is quite simple:

 1. Pick an alphabet of length $m$ and pick a matrix $M$ of size $n \times n$ that is invertible in $\mathbb{Z}_m$;
 2. Divide your message into groups of $n$ characters and map each character to a unique integer in $\mathbb{Z}_m$;
 3. Multiply each group of $n$ numerical values by your matrix $M$, obtaining a new set of $n$ numbers;
 4. Map all those numbers back to characters and put everything together again.

<a id="alphabet"></a>
### Alphabet

To pick an alphabet, one must take into account the type of messages we will be encoding, but it is also good to pay attention to something else: the size of the alphabet is the integer $m$ that will dictate in which $\mathbb{Z}_m$ we are working. If $m$ is a number with plenty of factors, then we will have a very hard time finding matrices $M$ that are invertible in $\mathbb{Z}_m$. To make our lifes easier, it is nice to consider an alphabet of prime length.

The usual alphabet has $26$ letters which is $3$ short of the prime number $29$; we can get there by adding the space, the comma and the period, for example. If we wanted, we could also take the exclamation and interrogation marks, to get an alphabet of length $31$ (also prime). We define the alphabet below.

In [134]:
ALPHABET = " abcdefghijklmnopqrstuvwxyz,.!?"
mod = len(ALPHABET)
print(mod)

31


<a id="strings_vectors"></a>
### Strings <-> numerical vectors

When we have a message `msg` to encrypt, we want to be able to turn all the characters in the string into unique numbers in $\mathbb{Z}_{31}$. A bit later we will also want to do the same thing the other way around, so I will just implement those two functions now.

In [135]:
lettermap = {ALPHABET[i]: i for i in range(mod)}
def aton(char: str) -> int:
    return lettermap[char]

def ntoa(n: int) -> str:
    return ALPHABET[n]

The two functions `aton` and `ntoa` above turn a single character into a number and a number into a single character, respectively. Notice that both functions assume that our request makes sense, so it will be up to us to be careful and only work with the right characters and numbers. For example, `aton('#')` gives an error:

In [136]:
aton('#')

KeyError: '#'

When encoding and decoding we will want to work with whole strings and lists of numbers so we will also get two functions to convert whole strings into lists of numbers and vice-versa.

In [137]:
def strtovec(string: str) -> list:
    return list(map(aton, string[::]))

def vectostr(vector: list) -> str:
    return "".join(list(map(ntoa, vector)))

In [138]:
strtovec("hello world!")

[8, 5, 12, 12, 15, 0, 23, 15, 18, 12, 4, 29]

In [139]:
vectostr(strtovec("hello world!"))

'hello world!'

<a id="#encoding"></a>
### Encoding

At this point we need to pick our $N$ that will define the size of the matrix. For simplicity we will pick $N = 2$, so we need a $2 \times 2$ matrix for $M$. Let us pick the key "cake" and let us be sure that it gives a valid matrix.

In [140]:
strtovec("cake")

[3, 1, 11, 5]

In [141]:
M = np.array([[3, 1], [11, 5]])
print(M)

[[ 3  1]
 [11  5]]


Now we need to ensure this matrix is invertible in $\mathbb{Z}_{31}$ which will be the case if the determinant of $M$ is not congruent to $0 \mod{31}$.

Computing determinants of $2\times 2$ matrices is fairly easy but we will be dealing with bigger matrices latter on, so we will just define a helper function.

In [142]:
def detmod(matrix: np.ndarray, mod: int) -> int:
    return int(np.linalg.det(matrix)) % mod

print(detmod(M, mod))

4


The final step to encoding is splitting the vector into bits of size $N$ and then multiplying by the matrix $M$. Say we had a string that was converted to the vector $(a_1, a_2, a_3, a_4, a_5, a_6)$; the next step is getting the vectors $v_1 = [a_1, a_2]^T$, $v_2 = [a_3, a_4]^T$ and $v_3 = [a_5, a_6]^T$ and computing $Mv_1$, $Mv_2$ and $Mv_3$. Because of the way matrices work, it is easier if we build a bigger matrix $V$ whose columns are the vectors $v_i$, and then compute $MV$.

$$V = \begin{bmatrix}v_1 & v_2 & v_3 \end{bmatrix} = \begin{bmatrix}a_1 & a_3 & a_5 \\ a_2 & a_4 & a_6 \end{bmatrix}$$

After computing that product, we finish it up by converting the numbers to letters and building the final matrix. Below we create a function that turns the list into a matrix and then, with that, we create a function that encrypts a string by using everything that we have built so far.

In [143]:
def vectomat(vector: list, N: int) -> np.ndarray:
    return np.array([[vector[N*i+j] for i in range(int(len(vector)/N))] for j in range(N)])

def mattovec(matrix: np.ndarray) -> list:
    length = matrix.size
    l = matrix.T.reshape([1, length]).tolist()
    return l[0]

print(strtovec("hello!"))
test1 = vectomat(strtovec("hello!"), 2)
print(test1)
print(strtovec("my name is rob"))
test2 = vectomat(strtovec("my name is rob"), 2)
print(test2)
print(mattovec(test2) == strtovec("my name is rob"))

[8, 5, 12, 12, 15, 29]
[[ 8 12 15]
 [ 5 12 29]]
[13, 25, 0, 14, 1, 13, 5, 0, 9, 19, 0, 18, 15, 2]
[[13  0  1  5  9  0 15]
 [25 14 13  0 19 18  2]]
True


In [144]:
def encrypt(msg: str, key_matrix: np.ndarray) -> str:
    # starting by finding the size of the matrix
    N = M.shape[0]
    # ensure the length of the message is a multiple of N
    while len(msg) % N:
        msg += " "
    # ensure only lower-case characters
    msg = msg.lower()
    matrix = vectomat(strtovec(msg), N)
    # below we CANNOT forget to take the matrix multiplication
    # in Z_m, i.e. we must use the % after multiplying
    encrypted = vectostr(mattovec(np.dot(key_matrix, matrix)%mod))
    return encrypted
    
print(encrypt("hello george!", M))
print(encrypt("the fox jumps over the lazy dog.", M))

!tqfnjziagziyi
floxbqjpt!xfzwe,butg!tl!!qm,,zr 


<a id="key_matrix"></a>
### Inverting the key matrix and decoding

When we get an encrypted message, to decrypt it we just need to invert the key matrix and repeat the process that we go through when encoding. That is, we could define the function `decrypt` as `decrypt(msg, key_matrix): encrypt(msg, inverse(key_matrix))`. This means we only need to know how to invert matrices! In the cell below I implement Gauss-Jordan elimination to do this, which you can read about in [here](https://www.mathportal.org/linear-algebra/matrices/gauss-jordan.php), for example.

In [145]:
def matrix_inversemod(matrix: np.ndarray, mod: int) -> np.ndarray:
    # use gauss-jordan to inverse the matrix
    # first use gauss to get an upper triangular matrix
    if matrix.ndim != 2:
        raise TypeError(f"Cannot invert tensor of rank {matrix.ndim}")
    elif matrix.shape[0] != matrix.shape[1]:
        raise TypeError(f"Cannot invert a non-square matrix")
    elif abs(detmod(matrix, mod)) < 0.00001:
        raise ValueError(f"Cannot invert a singular matrix")
    matrix = np.copy(matrix)
    size = matrix.shape[0]
    inverse = np.eye(size, dtype=int)
    matrix %= mod
    # start with Gaussian elimination, get upper triangular matrix
    # fix a column
    for col in range(0, size):
        # find a row that has a non-zero col-th element
        # this for always breaks because this row always exists
        for i in range(col, size):
            if matrix[i, col] != 0:
                break
        # swap the rows
        if i != col:
            matrix[[col, i], :] = matrix[[i, col], :]
            inverse[[col, i], :] = inverse[[i, col], :]
        # put a 1 in the position (col, col)
        inv = inversemod(matrix[col, col], mod)
        matrix[col, :] = inv*matrix[col, :] % mod
        inverse[col, :] = inv*inverse[col, :] % mod
        for row in range(col+1, size):
            piv = matrix[row, col]
            matrix[row, :] = (matrix[row, :] - piv*matrix[col, :]) % mod
            inverse[row, :] = (inverse[row, :] - piv*inverse[col, :]) % mod
    # now off to the Jordan part, get just a diagonal matrix
    # fix a column (right to left) and work from bottom to top
    for col in range(size-1, -1, -1):
        for row in range(col-1, -1, -1):
            piv = matrix[row, col]
            matrix[row, :] = (matrix[row, :] - piv*matrix[col, :]) % mod
            inverse[row, :] = (inverse[row, :] - piv*inverse[col, :]) % mod
    
    return inverse

M1 = np.array([[1,2,3],[3,2,1],[2,1,3]]) # some invertible matrix
print(np.dot(M1, matrix_inversemod(M1, mod))%mod)
print(np.dot(M, matrix_inversemod(M, mod))%mod)

[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[1 0]
 [0 1]]


With this function implemented, decoding messages becomes trivial!

In [146]:
def decrypt(msg: str, key_matrix: np.ndarray) -> str:
    return encrypt(msg, matrix_inversemod(key_matrix, mod))

encrypted = encrypt("the quick brown fox jumps over the lazy dog..?", M)
decrypted = decrypt(encrypted, M)
print(decrypted)

the quick brown fox jumps over the lazy dog..?


<a id="extras"></a>
### Extras

For the sake of whomever reads this and wants to experiment, I will just add another function that generates a random matrix that is invertible in a given modulo:

In [147]:
def random_invertible(dim: int, mod: int) -> np.ndarray:
    d = 0
    while not d:
        matrix = np.random.randint(0, mod, size=(dim,dim))
        d = detmod(matrix, mod)
    return matrix

print(random_invertible(2, mod))
print(random_invertible(5, mod))

[[13 24]
 [15 27]]
[[ 5 21 24 22  1]
 [20 27  4 16  2]
 [27 29  8 15 11]
 [ 4  4  1 17 19]
 [21 22  7 17 15]]


And two functions, one that turns words into key matrices and another one to do the reverse.

In [148]:
from math import sqrt
def word_to_key(word: str) -> np.ndarray:
    vec = strtovec(word)
    N = int(sqrt(len(vec)))
    return np.array(vec).reshape((N,N))

def key_to_word(matrix: np.ndarray) -> str:
    vec = matrix.reshape((1, matrix.size)).tolist()
    return "".join(vectostr(vec[0]))

print(word_to_key("cake"))
print(key_to_word(M))

[[ 3  1]
 [11  5]]
cake


<a id="hacking"></a>
Breaking the cipher
---------------------

Just because it is fun I will also include a small section on breaking the Hill Cipher. Because the way we encrypt messages is linear, the Hill Cipher is weak against plaintext attacks. This means that whenever we have an encrypted message and part of the original message, we can build a linear system and, by solving it, recover the key matrix!

Suppose we had the encrypted message '_qzj!xvutvhvwvv.m,dqij kggkuj..twualt.nsuwknv'_ that we happen to know that starts with '_the _'. We know that '_the _' $ \mapsto [20, 8, 5, 0]$ and that '_qzj!_' $ \mapsto [17, 26, 10, 29]$. Assuming the key matrix $M$ is a $2 \times 2$ matrix, we have that

$$M\begin{bmatrix} 20 \\ 8 \end{bmatrix} = \begin{bmatrix} 17 \\ 26 \end{bmatrix}$$

and that

$$M \begin{bmatrix} 5 \\ 0 \end{bmatrix} = \begin{bmatrix} 10 \\ 29 \end{bmatrix}$$

If we write the matrix $M$ explicitly in terms of its components,

$$M = \begin{bmatrix} a & b \\ c & d \end{bmatrix}$$

we can build the linear system

$$\begin{cases}
20a + 8b = 17 \\
20c + 8d = 26 \\
5a + 0b = 10 \\
5c + 0d = 29
\end{cases} \mod{m}$$

which, in turn, can be written as

$$\hat{M}\cdot x = \begin{bmatrix}
20 & 8 & 0 & 0 \\
0 & 0 & 20 & 8 \\
5 & 0 & 0 & 0 \\
0 & 0 & 5 & 0
\end{bmatrix}\cdot\begin{bmatrix} a \\ b \\ c \\ d \end{bmatrix} = \begin{bmatrix} 17 \\ 26 \\ 10 \\ 29\end{bmatrix} \mod{m}$$

If it happens that $\det\hat{M} \equiv 0 \mod{m}$, then we were unlucky and the correspondences we already have aren't enough. On the other hand, if the determinant of $\hat{M}$ isn't $0 \mod {m}$, we can invert $\hat{M}$, solve the linear system and then build $M$:

$$\hat{M} \cdot x = \begin{bmatrix} 17 \\ 26 \\ 10 \\ 29 \end{bmatrix} \iff x = \hat{M}^{-1}\cdot \begin{bmatrix}17 \\ 26 \\ 10 \\ 29 \end{bmatrix}$$

In [149]:
bigmatrix = np.array([[20, 8, 0, 0],
                        [0, 0, 20, 8],
                        [5, 0, 0, 0],
                        [0, 0, 5, 0]])
# check this is NOT equivalent to 0, i.e. bigmatrix is invertible
print(detmod(bigmatrix, mod))

vector = np.array([[17],[26],[10],[29]])
inverse = matrix_inversemod(bigmatrix, mod)

x = np.dot(inverse, vector)%mod

key_matrix = x.reshape((2,2))
print(key_matrix)
print(key_to_word(key_matrix))

# and now we decrypt the rest of the message
print(decrypt("qzj!xvutvhvwvv.m,dqij kggkuj..twualt.nsuwknv", key_matrix))

13
[[ 2  1]
 [12 12]]
ball
the quick brown fox jumps over the lazy dog 


Please notice that we assumed the key matrix was a $2 \times 2$ matrix just to make this simpler. The key matrix could be bigger and this method would still work. If the key matrix is $N \times N$, generally we need $N$ known correspondences to be able to build the $\hat{M}$ matrix and solve it for the components of $M$. If we are unlucky, we may need more than $N$ correspondences.

For those of you who are interested, I suggest that you build a function that does precisely what I just did in the cell above.

<a id="allcode"></a>
All the code
-------------

Below I will put everything together into a simple class `HillCipher`. This class will take the alphabet as initial argument and then it will have the methods needed to encrypt and decrypt messages, get invertible matrices to use as keys, etc. In the end a small usage example is provided. The cell below can be copied into a `.py` file, making the class accessible for future usage in your projects!

In [150]:
import numpy as np
from math import sqrt

class HillCipher(object):
    """This class will provide an interface for encrypting and decrypting
    messages through the Hill Cipher, a cipher that uses modular arithmetic
    and linear algebra to work"""
    def __init__(self, alphabet: str):
        # this isn't supposed to be changed throughout code execution
        self._alphabet = alphabet
        self._mod = len(self._alphabet)
        # build an auxiliar dictionary to make the ntoa function faster
        self._lettermap = {self._alphabet[i]: i for i in range(self._mod)}
        
    def aton(self, char: str) -> int:
        """Returns the numerical representation of char in the
        current alphabet; if the character doesn't exist, an
        error is thrown."""
        return self._lettermap[char]
    
    def ntoa(self, n: int) -> str:
        """Returns the character that is associated with the given
        integer in Z_{mod}. If the integer falls outside the legal
        range, an error is thrown."""
        return self._alphabet[n]
    
    def strtovec(self, string: str) -> list:
        """Turns a string into the corresponding list of integers"""
        return list(map(self.aton, string[::]))
    
    def vectostr(self, vector:list) -> str:
        """Turns a list of integers into the corresponding string"""
        return "".join(list(map(self.ntoa, vector)))

    def vectomat(self, vector:list, N: int) -> np.ndarray:
        """Auxiliar function that turns the numerical vector into a nicer
        representation for encoding/decoding"""
        return np.array([[vector[N*i+j] 
                          for i in range(int(len(vector)/N))] 
                             for j in range(N)])
    
    def mattovec(self, matrix: np.ndarray) -> list:
        """Auxiliar function that turns the matrix back into a 
        numerical vector"""
        length = matrix.size
        l = matrix.T.reshape([1, length]).tolist()
        return l[0]
    
    def is_valid(self, matrix: np.ndarray) -> bool:
        """Returns True if the matrix is a valid key matrix for the
        given alphabet; i.e. if it is invertible modulo the length of
        the internal alphabet"""
        return HillCipher.detmod(matrix, self._mod) != 0
    
    def encrypt(self, msg: str, key_matrix: np.ndarray) -> str:
        """encrypts the given message with the given key matrix"""
        # start by finding the size of the matrix
        N = key_matrix.shape[0]
        # ensure the length of the message is a multiple of N
        while len(msg) % N:
            msg += " "
        # ensure only lower-case characters
        msg = msg.lower()
        matrix = self.vectomat(strtovec(msg), N)
        # below we CANNOT forget to take the matrix multiplication
        # in Z_m, i.e. we must use the % after multiplying
        encrypted = self.vectostr(self.mattovec(np.dot(key_matrix, matrix)%self._mod))
        return encrypted
    
    def decrypt(self, msg: str, key_matrix: np.ndarray) -> str:
        """decrypts the given message that was encrypted with the given matrix"""
        return self.encrypt(msg, HillCipher.matrix_inversemod(key_matrix, mod))

    def word_to_key(self, word: str) -> np.ndarray:
        """Returns the key matrix corresponding to the given word"""
        vec = strtovec(word)
        N = int(sqrt(len(vec)))
        return np.array(vec).reshape((N,N))

    def key_to_word(self, matrix: np.ndarray) -> str:
        """Converts a key matrix into a word"""
        vec = matrix.reshape((1, matrix.size)).tolist()
        return "".join(self.vectostr(vec[0]))
    
    ### Auxiliar functions that deal with modular arithmetic
    def detmod(matrix: np.ndarray, mod: int) -> int:
        """Returns the determinant of the matrix in the given Z_{mod}"""
        return int(np.linalg.det(matrix)) % mod
        
    def inversemod(n: int, mod: int) -> int:
        """Finds the multiplicative inverse of n in Z_{mod}.
        When such inverse doesn't exist, 0 is returned."""
        for i in range(1, mod+1):
            if (n*i % mod) == 1:
                return i
        else:
            return 0
        
    def random_invertible(dim: int, mod: int) -> np.ndarray:
        """Returns an invertible matrix in the given Z_{mod}"""
        d = 0
        while not d:
            matrix = np.random.randint(0, mod, size=(dim,dim))
            d = HillCipher.detmod(matrix, mod)
        return matrix

    def matrix_inversemod(matrix: np.ndarray, mod: int) -> np.ndarray:
        """Auxiliar function to inverse a function in the given Z_{mod}"""
        # use gauss-jordan to inverse the matrix
        # first use gauss to get an upper triangular matrix
        if matrix.ndim != 2:
            raise TypeError(f"Cannot invert tensor of rank {matrix.ndim}")
        elif matrix.shape[0] != matrix.shape[1]:
            raise TypeError(f"Cannot invert a non-square matrix")
        elif abs(HillCipher.detmod(matrix, mod)) < 0.00001:
            raise ValueError(f"Cannot invert a singular matrix")
        matrix = np.copy(matrix)
        size = matrix.shape[0]
        inverse = np.eye(size, dtype=int)
        matrix %= mod
        # start with Gaussian elimination, get upper triangular matrix
        # fix a column
        for col in range(0, size):
            # find a row that has a non-zero col-th element
            # this for always breaks because this row always exists
            for i in range(col, size):
                if matrix[i, col] != 0:
                    break
            # swap the rows
            if i != col:
                matrix[[col, i], :] = matrix[[i, col], :]
                inverse[[col, i], :] = inverse[[i, col], :]
            # put a 1 in the position (col, col)
            inv = HillCipher.inversemod(matrix[col, col], mod)
            matrix[col, :] = inv*matrix[col, :] % mod
            inverse[col, :] = inv*inverse[col, :] % mod
            for row in range(col+1, size):
                piv = matrix[row, col]
                matrix[row, :] = (matrix[row, :] - piv*matrix[col, :]) % mod
                inverse[row, :] = (inverse[row, :] - piv*inverse[col, :]) % mod
        # now off to the Jordan part, get just a diagonal matrix
        # fix a column (right to left) and work from bottom to top
        for col in range(size-1, -1, -1):
            for row in range(col-1, -1, -1):
                piv = matrix[row, col]
                matrix[row, :] = (matrix[row, :] - piv*matrix[col, :]) % mod
                inverse[row, :] = (inverse[row, :] - piv*inverse[col, :]) % mod

        return inverse
    ### End of the modular arithmetic auxiliar functions
    
if __name__ == "__main__":
    abc = " abcdefghijklmnopqrstuvwxyz,.!?"
    hc = HillCipher(abc) # initialize the cipher
    
    # pick a key word, for example golf
    key_word = "golf"
    key_matrix = hc.word_to_key(key_word)
    print(f"The key word '{key_word}' is a valid key matrix: {hc.is_valid(key_matrix)}")
    
    # pick a random message to encrypt
    msg = "writing this jupyter notebook was fun, but moderately time consuming!"
    # encrypt the message
    encrypted = hc.encrypt(msg, key_matrix)
    print(f"the encrypted message reads:\n\t'{encrypted}'")
    
    # now we demonstrate hacking the hill cipher
    # assume we got the first 4 characters of the original message
    first_four = msg[:4]
    original4 = hc.strtovec(first_four)
    encrypted4 = hc.strtovec(encrypted[:4])
    print(f"we know the values {original4} have to be mapped to {encrypted4}")
    # build the big matrix, M hat
    Mhat = np.zeros((4, 4))
    Mhat[0, [0,1]] = original4[0:2]
    Mhat[1, [2,3]] = original4[0:2]
    Mhat[2, [0,1]] = original4[2:4]
    Mhat[3, [2,3]] = original4[2:4]
    print(f"the big matrix has determinant {HillCipher.detmod(Mhat, len(abc))}")
    
    # the big matrix is invertible, so invert it!
    x = np.dot(HillCipher.matrix_inversemod(Mhat, len(abc)), encrypted4)%len(abc)
    original_key_matrix = x.reshape((2,2))
    print(f"after solving the system, we got the following key word: {hc.key_to_word(original_key_matrix)}")
    print(f"the decrypted message reads:\n\t'{hc.decrypt(encrypted, original_key_matrix)}'")

The key word 'golf' is a valid key matrix: True
the encrypted message reads:
	'.lvkyfrvligez!ogjqzmxvbucjtvohucikplgt?lm ipjrzm.h!htu,ok nrepnmixqxqg'
we know the values [23, 18, 9, 20] have to be mapped to [28, 12, 22, 11]
the big matrix has determinant 11
after solving the system, we got the following key word: golf
the decrypted message reads:
	'writing this jupyter notebook was fun, but moderately time consuming! '
